In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt

import remap_values as rv
import good_plots   as gp

import cPickle

In [90]:
reload( rv )

<module 'remap_values' from 'remap_values.py'>

In [91]:
train_df = pd.read_csv('inp_data/train.csv')
test_df  = pd.read_csv('inp_data/test.csv')


train_df, test_df = rv.run_tan( train_df, test_df )

train_df = train_df.loc[ train_df['GrLivArea'] < 4000 ]

In [94]:
train_df, test_df = rv.norm_tan( train_df, test_df )

In [96]:
train_df['SalePrice'].head()

0    5.319106
1    5.258877
2    5.349278
3    5.146128
4    5.397940
Name: SalePrice, dtype: float64

In [3]:
big_df = train_df.drop( 'SalePrice', axis=1).append( test_df, ignore_index=True)

In [41]:
#PCA?
#Preprocess?

In [179]:
corr_series = train_df.corrwith( train_df['SalePrice'] ).sort_values( ascending=False )
print corr_series

SalePrice           1.000000
OverallQual         0.800837
AreaInside          0.729597
GrLivArea           0.720194
GarageCars          0.650456
TotalBsmtSF         0.647040
GarageArea          0.638182
NBath               0.636585
1stFlrSF            0.626261
NeighBin_4          0.574917
TotRmsAbvGrd        0.536751
YearBuilt           0.536327
YearRemodAdd        0.521238
MasVnrArea          0.474169
Fireplaces          0.466000
HeatingScale_4      0.441162
BsmtFinType1_6      0.436844
GarageFinish        0.421054
HasOpenPorchSF      0.420057
BsmtFinSF1          0.396542
HasMasVnrArea       0.380866
MSSubClass_60       0.373993
HasMasVnr           0.373428
PartialPlan         0.371902
RecentRemodel       0.355081
LotFrontage         0.338617
OpenPorchSF         0.330585
WoodDeckSF          0.329092
BsmtExposure_4      0.308133
HasWoodDeck         0.300442
                      ...   
BsmtFinType1_5     -0.103088
NewerDwelling_2    -0.114604
Fence_1            -0.115035
MSSubClass_160

In [180]:
col_list   = big_df.columns.values
large_list = []
for col in col_list:
    if ( big_df[col].unique().shape[0] > 2 ):
        print col, train_df[col].max()

1stFlrSF 3228
2ndFlrSF 1818
3SsnPorch 508
Age 138
AreaInside 3627
BedroomAbvGr 8
BsmtFinSF1 2188.0
BsmtFinSF2 1474.0
BsmtUnfSF 2336.0
EnclosedPorch 552
Fireplaces 3
GarageArea 1390.0
GarageCars 4.0
GarageYrBlt 2010.0
GrLivArea 3627
Id 1456
KitchenAbvGr 3
LotArea 215245
LotFrontage 313.0
LowQualFinSF 572
MSSubClass 190
MasVnrArea 1600.0
MiscVal 15500
MoSold 12
NBath 6.0
OpenPorchSF 547
OverallCond 9
OverallQual 10
PoolArea 738
SalePrice 625000
ScreenPorch 480
TimeSinceSold 4
TotRmsAbvGrd 14
TotalBsmtSF 3206.0
WoodDeckSF 857
YearBuilt 2010
YearRemodAdd 2010
YearSinceRemodel 60
YrSold 2010


In [181]:
col_list   = big_df.columns.values
large_list = []
for col in col_list:
    if ( big_df[col].unique().shape[0] < 3 ):
        print col, train_df[col].max()

Alley_0.0 1
Alley_1.0 1
Alley_2.0 1
BsmtExposure_0 1
BsmtExposure_1 1
BsmtExposure_2 1
BsmtExposure_3 1
BsmtExposure_4 1
BsmtFinType1_0 1
BsmtFinType1_1 1
BsmtFinType1_2 1
BsmtFinType1_3 1
BsmtFinType1_4 1
BsmtFinType1_5 1
BsmtFinType1_6 1
BsmtFinType2_0 1
BsmtFinType2_1 1
BsmtFinType2_2 1
BsmtFinType2_3 1
BsmtFinType2_4 1
BsmtFinType2_5 1
BsmtFinType2_6 1
Electrical 1
Fence_0 1
Fence_1 1
Fence_2 1
Fence_4 1
Foundation_0 1
Foundation_1 1
Foundation_2 1
Functional_0 0
Functional_2 1
Functional_3 1
Functional_4 1
Functional_5 1
Functional_6 1
Functional_7 1
Functional_8 1
GarageFinish 1
GarageType 1
Has2ndFlr 1
Has2ndFlrSF 1
Has3SsnPorch 1
HasEnclosedPorch 1
HasMasVnr 1
HasMasVnrArea 1
HasOpenPorchSF 1
HasScreenPorch 1
HasWoodDeck 1
HasWoodDeckSF 1
HeatingScale_0 1
HeatingScale_1 1
HeatingScale_2 1
HeatingScale_3 1
HeatingScale_4 1
HighSeason 1
LandSlope 1
LotConfig_0 1
LotConfig_1 1
LotConfig_2 1
LotConfig_3 1
LotConfig_4 1
MSSubClass_120 1
MSSubClass_150 0
MSSubClass_160 1
MSSubClass_1

In [182]:
invest = 'PoolQC'

In [183]:
big_df[invest].unique()

KeyError: 'PoolQC'

In [86]:
gp.plot_avg( train_df, invest, 'SalePrice' )

In [74]:
gp.hist_plot( big_df, invest )